# ERS

In [ ]:
import geopandas as gpd
import pandas as pd
import cartopy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import shapely

import shared_functions as sf

# settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

## Functions

In [ ]:
def annual_summary(df, size_col = 'size_GB'):
    count_size = (
        (df.groupby(['year']).agg(
        count = ('id','count'),
        size = (size_col, 'sum'),
        size_MB = ('size_MB','sum'),
        ))
        .reset_index()
        #.pivot(index='year', values=['count','size'])
        .fillna(0)
    )
    count_size.loc["Total"] = count_size.sum()
    count_size['count'] = count_size['count'].astype(int)
    count_size['mean_size_MB'] = count_size['size_MB']/count_size['count'] 
    count_size[size_col] = (count_size['size']).round(2)
    count_size = count_size.drop(columns=['size'])
    return count_size

## ERS-1 Level 1

In [ ]:
with open('metadata/ERS-1_LEVEL1_-50N_products.json') as f:
    data = json.load(f)
df_1 = pd.DataFrame.from_dict(data, orient='index')
df_1 = sf.preprocess_cmr_df(df_1, crs=3031, lon_first=True, aoi_filter=True)
print(df_1.shape)
print(df_1['time_start'].min(),df_1['time_start'].max())
df_1.head(2)

In [ ]:
annual_summary(df_1)

In [ ]:
title = 'ERS-1 complete level 1 spatial coverage (1994 - 1996)'
sf.plot_results_footprint_map(df_1, title=title)

## ERS-2 Level 1

In [ ]:
with open('metadata/ERS-2_LEVEL1_-50N_products.json') as f:
    data = json.load(f)
df_2 = pd.DataFrame.from_dict(data, orient='index')
df_2 = sf.preprocess_cmr_df(df_2, crs=3031, lon_first=True, aoi_filter=True)
print(df_2.shape)
print(df_2['time_start'].min(),df_2['time_start'].max())
df_2.head(2)

In [ ]:
title = 'ERS-2 complete level 1 spatial coverage (1995 - 2010)'
sf.plot_results_footprint_map(df_2, title=title)

In [ ]:
annual_summary(df_2)

## Both

In [ ]:
df = pd.concat([df_1,df_2])

In [ ]:
annual_summary(df)

In [ ]:
df.to_crs(4326).time_start.max()

In [ ]:
sf.plot_timeseries_products(df, 
	title=f'ERS-1 and ERS-2 (Level 1A) - Weekly Products' ,
	stack_col='dataset_id', 
	date_col='time_start',
	count_freq='7D', 
    plot_freq='2M')

In [ ]:
sf.plot_multiple_frequency(
    df,
    title = f"ERS 1/2 Revisit Frequency by Year",
    group='year', 
    sort_group='year',
    shape=(1500,1500),
    cbar_label='Revisit Count',
    n_cols=6,
)